In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)
running_order.head()

(6758, 20)

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,1,Stora Tuna OK 1,Stora Tuna OK,SWE,1,6.0,tilda östberg,1,Tilda Östberg,1.0,1.8857,0.0000,6.511986,1.881337,1.978520,0.048591,0.048591,1.8857,0.067659
1,1,1,Stora Tuna OK 1,Stora Tuna OK,SWE,2,5.7,magdalena olsson,2,Magdalena Olsson,6.0,1.9556,0.0886,6.992522,1.765439,1.961637,0.098099,0.098099,1.9556,0.088600
2,2,1,Stora Tuna OK 1,Stora Tuna OK,SWE,3,7.3,julia jakob,3,Julia Jakob,1.0,1.7824,0.0000,6.569075,1.798748,2.018901,0.110076,0.110076,1.7824,0.110076
3,3,1,Stora Tuna OK 1,Stora Tuna OK,SWE,4,7.9,tove alexandersson,4,Tove Alexandersson,6.0,1.8027,0.0586,6.290935,1.791712,1.973971,0.091129,0.091129,1.8027,0.067659
4,4,2,Göteborg Majorna OK 1,Göteborg Majorna OK,SWE,1,6.0,amanda hansson,1,Amanda Hansson,2.0,2.0053,0.0358,7.531656,1.880709,2.035837,0.077564,0.077564,2.0053,0.077564


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,1.8857,0.067659
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.9556,0.088600
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.7824,0.110076
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.8027,0.067659
4,2,Göteborg Majorna OK 1,1,6.0,Amanda Hansson,2.0053,0.077564


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
1              Stora Tuna OK 1        Stora Tuna OK 1        Stora Tuna OK 1   
2        Göteborg Majorna OK 1  Göteborg Majorna OK 1  Göteborg Majorna OK 1   
3                   MS Parma 1             MS Parma 1             MS Parma 1   
4          Järla Orientering 1    Järla Orientering 1    Järla Orientering 1   
5               Alfta ÖSA OK 1         Alfta ÖSA OK 1         Alfta ÖSA OK 1   

                               dist                               name  \
leg                          4    1    2    3    4                   1   
team_id                                                                  
1              Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
2        Göteborg Majorna OK 1  6.0  5.7  7.3  7.9      Amanda Hansson   
3                   MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4          Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5               Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   

                                                                       \
leg                        2                    3                   4   
team_id                                                                 
1           Magdalena Olsson          Julia Jakob  Tove Alexandersson   
2              Runa Fremstad     Kristine Berglia          Lisa Holer   
3             Anni Heikkonen       Tiia Istolahti       Maija Sianoja   
4        Anna Dahlgren-Rosén  Elin Hemmyr-Skantze      Sofie Bachmann   
5            Josefine Heikka         Irina Nyberg    Natalia Gemperle   

        log_mean                            log_std                      \
leg            1        2       3       4         1         2         3   
team_id                                                                   
1         1.8857  1.95560  1.7824  1.8027  0.067659  0.088600  0.110076   
2         2.0053  2.00810  1.9581  1.9854  0.077564  0.087900  0.067659   
3         1.8899  1.94010  1.8669  1.9157  0.067659  0.108600  0.128204   
4         1.9748  1.94591  1.8086  1.9289  0.112800  0.182322  0.124320   
5         1.8949  1.88830  1.9000  1.8720  0.110100  0.117383  0.095300   

                   
leg             4  
team_id            
1        0.067659  
2        0.097276  
3        0.067659  
4        0.087344  
5        0.125945

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1697, 20)

(1681, 20)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,34.589841,71.674661,111.359357,157.855858,45.300946,89.351075,137.174110,186.725193,39.562341,80.058378,...,34.935729,42.035753,45.300946,47.942873,53.995565,54.692577,39.562341,40.438336,43.316150,47.945860
2,38.395554,77.711721,127.462321,181.108504,52.134712,97.944279,151.881234,213.957120,44.594984,87.080070,...,45.191923,47.634863,52.134712,50.311953,59.092216,69.237595,44.594984,42.384102,51.755612,57.571184
3,34.690247,70.313660,112.626425,164.780655,45.434300,90.259200,143.661753,199.120132,39.672531,79.435441,...,36.695506,47.038588,45.434300,49.140157,60.628375,61.185843,39.672531,39.669120,47.241794,53.807580
4,34.806580,68.197197,109.594541,162.065691,54.011219,103.458684,151.361342,207.842149,43.202917,83.493853,...,34.767184,45.742626,54.011219,57.202854,56.840581,64.362887,43.202917,39.868824,44.462335,54.342633
5,32.282259,66.521509,112.364604,159.602687,49.476968,91.143422,143.288443,200.049284,39.975928,77.787719,...,40.366005,40.314013,49.476968,47.316135,58.855203,65.265563,39.975928,37.628307,48.823132,51.266746


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,name_3,name_4,log_mean_1,log_mean_2,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4
team_id,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,Julia Jakob,Tove Alexandersson,1.89,1.96,1.78,1.80,0.07,0.09,0.11,0.07
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,Kristine Berglia,Lisa Holer,2.01,2.01,1.96,1.99,0.08,0.09,0.07,0.10
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,Tiia Istolahti,Maija Sianoja,1.89,1.94,1.87,1.92,0.07,0.11,0.13,0.07
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,Elin Hemmyr-Skantze,Sofie Bachmann,1.97,1.95,1.81,1.93,0.11,0.18,0.12,0.09
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,Irina Nyberg,Natalia Gemperle,1.89,1.89,1.90,1.87,0.11,0.12,0.10,0.13


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,34.935729,42.035753,45.300946,47.942873,53.995565,54.692577,39.562341,40.438336,43.316150,47.945860
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,45.191923,47.634863,52.134712,50.311953,59.092216,69.237595,44.594984,42.384102,51.755612,57.571184
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,36.695506,47.038588,45.434300,49.140157,60.628375,61.185843,39.672531,39.669120,47.241794,53.807580
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,34.767184,45.742626,54.011219,57.202854,56.840581,64.362887,43.202917,39.868824,44.462335,54.342633
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.366005,40.314013,49.476968,47.316135,58.855203,65.265563,39.975928,37.628307,48.823132,51.266746


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9556   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.8027   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   
        2    Göteborg Majorna OK 1   5.7       Runa Fremstad    2.0081   
        3    Göteborg Majorna OK 1   7.3    Kristine Berglia    1.9581   

              log_std  fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                              
1       1    0.067659    34.589841   45.300946   39.562341   39.636704   
        2    0.088600    71.674661   89.351075   80.058378   80.194171   
        3    0.110076   111.359357  137.174110  123.642593  123.775494   
        4    0.067659   157.855858  186.725193  171.634435  171.831014   
2       1    0.077564    38.395554   52.134712   44.594984   44.748456   
        2    0.087900    77.711721   97.944279   87.080070   87.329343   
        3    0.067659   127.462321  151.881234  139.052655  139.155177   

             fin_sum_log_mean  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg                                                                   
1       1            3.677420         0.068346         0.0              0.0   
        2            4.382863         0.056358         0.0              0.0   
        3            4.817047         0.053323         0.0              0.0   
        4            5.145596         0.042771         0.0              0.0   
2       1            3.798018         0.077911         0.0              0.0   
        2            4.467944         0.059013         0.0              0.0   
        3            4.934592         0.044670         0.0              0.0   

             personal_start95  personal_end95  personal_median  
team_id leg                                                     
1       1           34.589841       45.300946        39.562341  
        2           33.877420       47.942873        40.438336  
        3           34.935729       53.995565        43.316150  
        4           42.035753       54.692577        47.945860  
2       1           38.395554       52.134712        44.594984  
        2           35.861632       50.311953        42.384102  
        3           45.191923       59.092216        51.755612

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9556   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.8027   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   

              log_std  fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                              
1       1    0.067659    34.589841   45.300946   39.562341   39.636704   
        2    0.088600    71.674661   89.351075   80.058378   80.194171   
        3    0.110076   111.359357  137.174110  123.642593  123.775494   
        4    0.067659   157.855858  186.725193  171.634435  171.831014   
2       1    0.077564    38.395554   52.134712   44.594984   44.748456   

             fin_sum_log_mean  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg                                                                   
1       1            3.677420         0.068346         0.0              0.0   
        2            4.382863         0.056358         0.0              0.0   
        3            4.817047         0.053323         0.0              0.0   
        4            5.145596         0.042771         0.0              0.0   
2       1            3.798018         0.077911         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           34.589841       45.300946        39.562341   
        2           33.877420       47.942873        40.438336   
        3           34.935729       53.995565        43.316150   
        4           42.035753       54.692577        47.945860   
2       1           38.395554       52.134712        44.594984   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:39:33.740438700 2019-06-15 14:34:35.390475512   
        2   2019-06-15 15:20:03.502677917 2019-06-15 15:11:40.479667187   
        3   2019-06-15 16:03:38.555583715 2019-06-15 15:51:21.561432362   
        4   2019-06-15 16:51:38.066092730 2019-06-15 16:37:51.351467133   
2       1   2019-06-15 14:44:35.699014187 2019-06-15 14:38:23.733258486   

                           fin_time_end95  personal_estimate_interval  
team_id leg                                                            
1       1   2019-06-15 14:45:18.056744576                   10.711104  
        2   2019-06-15 15:29:21.064472436                   14.065454  
        3   2019-06-15 16:17:10.446616411                   19.059835  
        4   2019-06-15 17:06:43.511601447                   12.656824  
2       1   2019-06-15 14:52:08.082729578                   13.739158

In [16]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

#for_print[for_print['team'].str.contains("Stadin")]
for_print = for_print[for_print['team'].str.contains("Jyry")]

for_print = for_print[[
 'leg',
 'name', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,
341,1,Jola Maisala,41.86,54.17,69.83,27.97
341,2,Kaija Rantoja,45.14,58.53,75.57,30.43
341,3,Jenni Castren,56.94,75.39,99.00,42.06
341,4,Inka Lemmetyinen,55.33,78.42,111.73,56.41
528,1,Päivi Taponen,47.01,63.86,86.50,39.49
528,2,Mervi Luurila,47.28,67.64,96.24,48.96
528,3,Suvi Tiainen,71.72,104.18,149.61,77.89
528,4,Tiina Isoniemi,59.11,68.15,78.64,19.53
740,1,Mila Viksilä,55.73,79.25,113.43,57.70


In [17]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
1332,341,1,Rasti-Jyry 1,Jola Maisala,2.200700,0.130900,3.992107,0.130423,2019-06-15 11:41:51.596704960+00:00,2019-06-15 12:09:50.082359076+00:00,2019-06-15 11:54:10.055367231+00:00,0.0000,0.0
1333,341,2,Rasti-Jyry 1,Kaija Rantoja,2.328600,0.130600,4.729369,0.092238,2019-06-15 12:34:19.552264452+00:00,2019-06-15 13:15:46.478183985+00:00,2019-06-15 12:53:18.616622448+00:00,0.0000,0.0
1334,341,3,Rasti-Jyry 1,Jenni Castren,2.334200,0.141700,5.243004,0.078788,2019-06-15 13:42:24.291464567+00:00,2019-06-15 14:40:43.498324156+00:00,2019-06-15 14:09:10.513187407+00:00,0.0000,0.0
1335,341,4,Rasti-Jyry 1,Inka Lemmetyinen,2.293200,0.178309,5.593699,0.076837,2019-06-15 14:51:13.643646002+00:00,2019-06-15 16:12:10.149468422+00:00,2019-06-15 15:28:37.742220163+00:00,0.0000,0.0
2076,528,1,Rasti-Jyry 2,Päivi Taponen,2.361600,0.156778,4.157126,0.154389,2019-06-15 11:47:00.755916119+00:00,2019-06-15 12:26:29.969465017+00:00,2019-06-15 12:03:51.504212617+00:00,0.0000,0.0
2077,528,2,Rasti-Jyry 2,Mervi Luurila,2.473200,0.181082,4.885732,0.119290,2019-06-15 12:45:04.680818081+00:00,2019-06-15 13:48:15.214181423+00:00,2019-06-15 13:12:25.258338928+00:00,0.0000,0.0
2078,528,3,Rasti-Jyry 2,Suvi Tiainen,2.655400,0.187373,5.471945,0.106555,2019-06-15 14:13:30.631700993+00:00,2019-06-15 15:52:39.586770773+00:00,2019-06-15 14:57:47.747593164+00:00,0.0000,0.0
2079,528,4,Rasti-Jyry 2,Tiina Isoniemi,2.155400,0.073500,5.725161,0.084357,2019-06-15 15:20:43.317546844+00:00,2019-06-15 17:02:04.966052532+00:00,2019-06-15 16:06:22.699744463+00:00,0.1176,0.0
2912,740,1,Rasti-Jyry 4,Mila Viksilä,2.581289,0.182322,4.374855,0.182593,2019-06-15 11:55:43.865870953+00:00,2019-06-15 12:53:25.709465265+00:00,2019-06-15 12:19:14.911597252+00:00,0.0000,0.0
2913,740,2,Rasti-Jyry 4,Tiina Sidensnöre,2.275900,0.154400,4.913196,0.124935,2019-06-15 12:46:37.027787685+00:00,2019-06-15 13:54:10.908696175+00:00,2019-06-15 13:15:41.384184599+00:00,0.0000,0.0


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [19]:
for_web.shape

(6724, 13)

In [20]:
np.exp([-0.075241, 0.0, 0.01, 0.5])

array([0.92751993, 1.        , 1.01005017, 1.64872127])